In [1]:
from keras.applications import VGG16
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Lambda
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


**Use ImageDataGenerator for data augmentation and resizing**

In [2]:
train_dir = 'data/faces/train'
val_dir = 'data/faces/val'
test_dir = 'data/test'
batch_size = 32

def generate(generator, directory, classes, batch_size):
    return generator.flow_from_directory(directory,
                                         target_size=(112,112),
                                         classes=classes,
                                         class_mode='categorical',
                                         batch_size=batch_size,
                                         shuffle=True,
                                         color_mode='grayscale')

def generate_augmented_batches(directory, batch_size=32, classes=None):
    gen = ImageDataGenerator(rotation_range=15,
                             width_shift_range=0.07,
                             height_shift_range=0.07,
                             shear_range=0.03,
                             horizontal_flip=True,)
    return generate(gen, directory, classes, batch_size)

def generate_batches(directory, batch_size=32, classes=None):
    gen = ImageDataGenerator()
    return generate(gen, directory, classes, batch_size)

In [3]:
train_batches = generate_augmented_batches(train_dir, batch_size)
val_batches = generate_batches(val_dir, batch_size)

Found 14961 images belonging to 3 classes.
Found 1214 images belonging to 3 classes.


In [4]:
mean_pixel_value = 126.945034642

In [5]:
def zero_center_fn(x):
    return x - mean_pixel_value

def add_conv_block(model, layers, filters):
    for _ in range(layers):
        model.add(Conv2D(filters, 3, padding='same', activation='relu'))
        model.add(BatchNormalization())
    model.add(MaxPooling2D())
    
def add_dense_block(model, layers, nodes, dropout):
    for _ in range(layers):
        model.add(Dense(nodes))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
    
def vgg_like(dropout=0.5, zero_center=False):
    model = Sequential()
    
    if(zero_center):
        model.add(Lambda(zero_center_fn, input_shape=(112,112,1)))
        add_conv_block(model, 2, 32)
    else:
        model.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=(112,112,1)))
        model.add(BatchNormalization())
        add_conv_block(model, 1, 32)
        
    add_conv_block(model, 1, 64)
    add_conv_block(model, 2, 128)
    add_conv_block(model, 2, 256)
    model.add(Flatten())
    add_dense_block(model, 2, 4096, dropout)
    model.add(Dense(3, activation='softmax'))
    
    return model

In [6]:
def train(model,
          training_batches,
          validation_batches,
          model_filename,
          batch_size=64,
          epochs=100,
          lr=5e-4,
          patience=10,
          min_lr=1e-7,
          reduce_factor=0.5,
          log_dir='/home/ubuntu/nbs/tensorboard_logs',
          initial_epoch=0):
    model.compile(optimizer=Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Add various callbacks: checkpointing on best validation loss, tensorboard for viewing loss function
    # and learning rate decay for dropping learning rate when learning has hit a plateau.
    checkpoint = ModelCheckpoint(model_filename, save_best_only=True, monitor='val_loss')
    tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)
    lrdecay = ReduceLROnPlateau(monitor='val_loss', factor=reduce_factor, patience=patience, min_lr=min_lr)
    
    history2 = model.fit_generator(training_batches,
                                  steps_per_epoch=(14961 // batch_size),
                                  validation_data=validation_batches,
                                  validation_steps=(1214 // batch_size),
                                  callbacks=[checkpoint, tensorboard, lrdecay],
                                  epochs=epochs,
                                  initial_epoch=initial_epoch)

In [7]:
model = vgg_like(dropout=0.875, zero_center=True)

In [11]:
model_file_path = 'models/vgg_like_dp875/vgg.{epoch:02d}-{val_loss:.4f}.hdf5'
train(model, train_batches, val_batches, model_file_path, epochs=120, batch_size=batch_size)

INFO:tensorflow:Summary name conv2d_8/kernel:0 is illegal; using conv2d_8/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_8/bias:0 is illegal; using conv2d_8/bias_0 instead.
INFO:tensorflow:Summary name batch_normalization_10/gamma:0 is illegal; using batch_normalization_10/gamma_0 instead.
INFO:tensorflow:Summary name batch_normalization_10/beta:0 is illegal; using batch_normalization_10/beta_0 instead.
INFO:tensorflow:Summary name batch_normalization_10/moving_mean:0 is illegal; using batch_normalization_10/moving_mean_0 instead.
INFO:tensorflow:Summary name batch_normalization_10/moving_variance:0 is illegal; using batch_normalization_10/moving_variance_0 instead.
INFO:tensorflow:Summary name conv2d_9/kernel:0 is illegal; using conv2d_9/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_9/bias:0 is illegal; using conv2d_9/bias_0 instead.
INFO:tensorflow:Summary name batch_normalization_11/gamma:0 is illegal; using batch_normalization_11/gamma_0 instead.
INFO:tensorflow:Summ

KeyboardInterrupt: 

**Load Best Model**

In [8]:
model.load_weights('models/vgg_like_dp875/vgg.58-0.4157.hdf5')

**Load Images**

In [9]:
test_images = np.genfromtxt('test_data.csv', delimiter=',').reshape((-1,48,48)).astype(np.uint8)
nb_images = test_images.shape[0]

**Save images as jpgs so we can exploit ImageDataGenerator for resizing.**

In [10]:
from PIL import Image
test_dir = 'data/faces/test/'
for i in range(nb_images):
    img = Image.fromarray(test_images[i])
    img.save(test_dir + 'unknown/' + str(i) + '.jpg', 'JPEG')

In [21]:
test_batches = generate_batches(test_dir, classes=['unknown'], batch_size=5)
nb_batches = 3965 / 5

Found 3965 images belonging to 1 classes.


In [22]:
predictions = np.argmax(model.predict_generator(test_batches, nb_batches), axis=1)

**Write predictions to csv file**

In [54]:
filenames = test_batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
sorted_idx = np.argsort(ids)

In [40]:
sub_file_name = 'vgg_like.csv'
submission_matrix = np.stack([ids[sorted_idx], predictions[sorted_idx]], axis=1)
np.savetxt(sub_file_name, submission_matrix, delimiter=',', fmt='%d,%d', header='Id,Category')